In [ ]:
!pip install datasets
!pip install gensim==4.3.1
!pip install tqdm
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 5.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/cola

In [ ]:
from datasets import load_dataset
import pandas as pd
import json
import os
import re
from google.colab import drive
from tqdm import tqdm
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import nltk
import torch
import math
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from scipy.sparse import hstack
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/AI")

Mounted at /content/drive


In [ ]:
hc3_df = pd.read_json(path_or_buf="reddit_eli5.jsonl", lines=True)
hc3_df

,question,human_answers,chatgpt_answers,index
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN
...,...,...,...,...
17107,Why is the general Xbox One reveal feedback ne...,[A few items people do n't seem to like : 1 . ...,[It's difficult to say exactly why the general...,NaN
17108,If infrared is thermal why does n't my remote ...,"[It 's very , very low power . Around the orde...",[Infrared light is a type of light that we can...,NaN
17109,Why are cars all about their brake - horse pow...,[Brake horsepower has nothing to do with brake...,"[Cars are not just about how fast they can go,...",NaN
17110,"What does a CO^2 scrubber do , and how does it...",[A [ scrubber ] ( URL_0 ) is a engineering pro...,[A CO2 scrubber is a device that removes carbo...,NaN


In [ ]:
# drop index col which is all NaN
processed_hc3_df = hc3_df.drop(["index"], axis=1)
processed_hc3_df.dropna(inplace=True)
processed_hc3_df

,question,human_answers,chatgpt_answers
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...
...,...,...,...
17107,Why is the general Xbox One reveal feedback ne...,[A few items people do n't seem to like : 1 . ...,[It's difficult to say exactly why the general...
17108,If infrared is thermal why does n't my remote ...,"[It 's very , very low power . Around the orde...",[Infrared light is a type of light that we can...
17109,Why are cars all about their brake - horse pow...,[Brake horsepower has nothing to do with brake...,"[Cars are not just about how fast they can go,..."
17110,"What does a CO^2 scrubber do , and how does it...",[A [ scrubber ] ( URL_0 ) is a engineering pro...,[A CO2 scrubber is a device that removes carbo...


# Clean Dataset

In [ ]:
# keep rows with non empty chatgpt ans
has_chatgpt_ans = processed_hc3_df["chatgpt_answers"].map(lambda ans_lst:ans_lst!=[])
processed_hc3_df = processed_hc3_df[has_chatgpt_ans]

#Need to expand the answers and chatgpt ans -- as for one question, there can be several human answers obtained
row_list = []
for index, row in processed_hc3_df.iterrows():
  human_ans_list = row["human_answers"]
  chatgpt_ans_list = row["chatgpt_answers"]
  # note that sometimes there can be 2 human ans, and 1 chatgpt ans for e.g. 
  num_chatgpt_ans = len(chatgpt_ans_list)
  for i in range(len(human_ans_list)):
    human_ans = human_ans_list[i]
    if i >= num_chatgpt_ans:
      chatgpt_ans = chatgpt_ans_list[num_chatgpt_ans-1]
    else:
      chatgpt_ans = chatgpt_ans_list[i]
    row_list.append([row["question"],human_ans,chatgpt_ans])

processed_hc3_df = pd.DataFrame(row_list, columns = ["question", "human_answer", "chatgpt_answer"])
print(processed_hc3_df.shape)
processed_hc3_df.head()

(49971, 3)


,question,human_answer,chatgpt_answer
0,"Why is every book I hear about a "" NY Times # ...","Basically there are many categories of "" Best ...",There are many different best seller lists tha...
1,"Why is every book I hear about a "" NY Times # ...","If you 're hearing about it , it 's because it...",There are many different best seller lists tha...
2,"Why is every book I hear about a "" NY Times # ...","One reason is lots of catagories . However , h...",There are many different best seller lists tha...
3,"If salt is so bad for cars , why do we use it ...",salt is good for not dying in car crashes and ...,Salt is used on roads to help melt ice and sno...
4,"If salt is so bad for cars , why do we use it ...","In Minnesota and North Dakota , they tend to u...",Salt is used on roads to help melt ice and sno...


# Convert the shape of the df to n x 2, where the first column is the response and the second column is the label.

In [ ]:
chatgpt_responses = processed_hc3_df.loc[:, ["chatgpt_answer"]]
chatgpt_responses["label"]=1
chatgpt_responses.rename(columns = {'chatgpt_answer':'response'}, inplace = True)
chatgpt_responses.drop_duplicates(inplace=True)
print(chatgpt_responses.shape)
chatgpt_responses.head()

(16122, 2)


,response,label
0,There are many different best seller lists tha...,1
3,Salt is used on roads to help melt ice and sno...,1
6,There are a few reasons why we still have SD (...,1
9,It is generally not acceptable or ethical to a...,1
12,After the Wright Brothers made the first power...,1


In [ ]:
human_responses = processed_hc3_df.loc[:, ["human_answer"]]
human_responses["label"]=0
human_responses.rename(columns = {'human_answer':'response'}, inplace = True)
human_responses.drop_duplicates(inplace=True)
print(human_responses.shape)
human_responses.head()

(45509, 2)


,response,label
0,"Basically there are many categories of "" Best ...",0
1,"If you 're hearing about it , it 's because it...",0
2,"One reason is lots of catagories . However , h...",0
3,salt is good for not dying in car crashes and ...,0
4,"In Minnesota and North Dakota , they tend to u...",0


In [ ]:
# sample the same number of human responses as chatgpt responses to make type equal in number
sampled_human_responses = human_responses.sample(n = chatgpt_responses.shape[0])
sampled_human_responses

,response,label
9298,They have a powerful and well organized lobby ...,0
32814,[ The second law of themodynamics ] ( URL_0 ) ...,0
38047,"Their "" employer "" is the taxpayer , and the t...",0
8367,I could think of some reasons why but they may...,0
47992,Most of it has to do with casting . The castin...,0
...,...,...
43793,Watch this [ video from Google regarding the d...,0
29527,Have you ever had the experience of getting fi...,0
48384,Say you have a rare pokemon card you do n't re...,0
852,They are work clothes . I 'm sure they probabl...,0


In [ ]:
Corpus = pd.concat([chatgpt_responses, sampled_human_responses], ignore_index=True)
print(Corpus.shape)
Corpus.head()

(32244, 2)


,response,label
0,There are many different best seller lists tha...,1
1,Salt is used on roads to help melt ice and sno...,1
2,There are a few reasons why we still have SD (...,1
3,It is generally not acceptable or ethical to a...,1
4,After the Wright Brothers made the first power...,1


In [ ]:
# Step - a : Remove blank rows if any.
Corpus['response'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['response'] = [entry.lower() for entry in Corpus['response']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['response']= [word_tokenize(entry) for entry in Corpus['response']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in tqdm(enumerate(Corpus['response'])):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

32244it [16:28, 32.61it/s]


In [ ]:
Corpus.head()

,response,label,text_final
0,"[there, are, many, different, best, seller, li...",1,"['many', 'different', 'best', 'seller', 'list'..."
1,"[salt, is, used, on, roads, to, help, melt, ic...",1,"['salt', 'use', 'road', 'help', 'melt', 'ice',..."
2,"[there, are, a, few, reasons, why, we, still, ...",1,"['reason', 'still', 'sd', 'standard', 'definit..."
3,"[it, is, generally, not, acceptable, or, ethic...",1,"['generally', 'acceptable', 'ethical', 'advoca..."
4,"[after, the, wright, brothers, made, the, firs...",1,"['wright', 'brother', 'make', 'first', 'powere..."


In [ ]:
Corpus.dropna()
Corpus

,response,label,text_final
0,"[there, are, many, different, best, seller, li...",1,"['many', 'different', 'best', 'seller', 'list'..."
1,"[salt, is, used, on, roads, to, help, melt, ic...",1,"['salt', 'use', 'road', 'help', 'melt', 'ice',..."
2,"[there, are, a, few, reasons, why, we, still, ...",1,"['reason', 'still', 'sd', 'standard', 'definit..."
3,"[it, is, generally, not, acceptable, or, ethic...",1,"['generally', 'acceptable', 'ethical', 'advoca..."
4,"[after, the, wright, brothers, made, the, firs...",1,"['wright', 'brother', 'make', 'first', 'powere..."
...,...,...,...
66727,"[your, blood, contains, a, clotting, factor, t...",0,"['blood', 'contain', 'clotting', 'factor', 'ca..."
66728,"[lot, 's, of, people, without, a, job, ., the,...",0,"['lot', 'people', 'without', 'job', 'unemploym..."
66729,"[gndn, 's, joke, is, very, on, point, to, your...",0,"['gndn', 'joke', 'point', 'question', 'test', ..."
66730,"[the, statement, ``, the, universe, is, infini...",0,"['statement', 'universe', 'infinite', 'mean', ..."


# Save the processed dataset

In [ ]:
Corpus.to_csv("processed_dataset.csv", index=False)

# Resume using the saved dataset

In [ ]:
Corpus = pd.read_csv("processed_dataset.csv")
Corpus

,response,label,text_final
0,"['there', 'are', 'many', 'different', 'best', ...",1,"['many', 'different', 'best', 'seller', 'list'..."
1,"['salt', 'is', 'used', 'on', 'roads', 'to', 'h...",1,"['salt', 'use', 'road', 'help', 'melt', 'ice',..."
2,"['there', 'are', 'a', 'few', 'reasons', 'why',...",1,"['reason', 'still', 'sd', 'standard', 'definit..."
3,"['it', 'is', 'generally', 'not', 'acceptable',...",1,"['generally', 'acceptable', 'ethical', 'advoca..."
4,"['after', 'the', 'wright', 'brothers', 'made',...",1,"['wright', 'brother', 'make', 'first', 'powere..."
...,...,...,...
66727,"['your', 'blood', 'contains', 'a', 'clotting',...",0,"['blood', 'contain', 'clotting', 'factor', 'ca..."
66728,"['lot', ""'s"", 'of', 'people', 'without', 'a', ...",0,"['lot', 'people', 'without', 'job', 'unemploym..."
66729,"['gndn', ""'s"", 'joke', 'is', 'very', 'on', 'po...",0,"['gndn', 'joke', 'point', 'question', 'test', ..."
66730,"['the', 'statement', '``', 'the', 'universe', ...",0,"['statement', 'universe', 'infinite', 'mean', ..."


# Split after vectorizing

In [ ]:
Encoder = LabelEncoder()
Y = Encoder.fit_transform(Corpus['label'])
Y

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
X_Tfidf = Tfidf_vect.transform(Corpus['text_final'])
X_Tfidf

<32244x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 1373917 stored elements in Compressed Sparse Row format>

In [ ]:
print(Tfidf_vect.vocabulary_)

{'many': 2709, 'different': 1249, 'best': 437, 'seller': 3957, 'list': 2615, 'publish': 3535, 'various': 4795, 'organization': 3105, 'new': 2979, 'york': 4993, 'time': 4556, 'one': 3073, 'rank': 3590, 'book': 498, 'united': 4726, 'state': 4253, 'base': 390, 'sale': 3881, 'data': 1108, 'number': 3022, 'retailer': 3783, 'newspaper': 2981, 'widely': 4928, 'consider': 924, 'influential': 2291, 'industry': 2276, 'important': 2238, 'note': 3006, 'category': 655, 'country': 1023, 'possible': 3380, 'could': 1018, 'another': 210, 'additionally': 75, 'term': 4487, 'often': 3066, 'use': 4766, 'refer': 3658, 'sell': 3956, 'well': 4911, 'regardless': 3675, 'whether': 4921, 'specific': 4196, 'may': 2745, 'hear': 2085, 'even': 1585, 'specifically': 4197, 'salt': 3883, 'road': 3820, 'help': 2099, 'melt': 2772, 'ice': 2193, 'snow': 4127, 'improve': 2244, 'traction': 4606, 'winter': 4946, 'month': 2876, 'cold': 805, 'outside': 3124, 'water': 4887, 'freeze': 1875, 'make': 2683, 'dangerous': 1104, 'car': 

## Split training data

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(X_Tfidf,Y,test_size=0.3)

In [ ]:
Train_X

<46712x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2105828 stored elements in Compressed Sparse Row format>

In [ ]:
Test_X

<20020x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 905566 stored elements in Compressed Sparse Row format>

In [ ]:
Train_Y

array([1, 0, 0, ..., 0, 1, 0])

In [ ]:
Test_Y

array([1, 0, 1, ..., 1, 0, 0])

# Linear kernel SVM

In [ ]:
SVM = svm.SVC(C=1.0, kernel='linear')
SVM.fit(Train_X,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print(classification_report(Test_Y, predictions_SVM))

SVM Accuracy Score ->  92.46433739921439
              precision    recall  f1-score   support

           0       0.93      0.92      0.92      4768
           1       0.92      0.93      0.93      4906

    accuracy                           0.92      9674
   macro avg       0.92      0.92      0.92      9674
weighted avg       0.92      0.92      0.92      9674



## Save the model

In [ ]:
# save the model to disk
import pickle
with open('svm_linear_model.pkl','wb') as f:
    pickle.dump(SVM,f)

# RBF kernel SVM

In [ ]:
RBF_SVM = svm.SVC(C=1.0, kernel='rbf')
RBF_SVM.fit(Train_X,Train_Y)
# predict the labels on validation dataset
predictions_RBF_SVM = RBF_SVM.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_RBF_SVM, Test_Y)*100)
print(classification_report(Test_Y, predictions_RBF_SVM))

SVM Accuracy Score ->  93.34298118668596
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      4768
           1       0.94      0.93      0.93      4906

    accuracy                           0.93      9674
   macro avg       0.93      0.93      0.93      9674
weighted avg       0.93      0.93      0.93      9674



## Save the model

In [ ]:
# save the model to disk
import pickle
with open('svm_rbf_model.pkl','wb') as f:
    pickle.dump(RBF_SVM,f)

# Sigmoid kernel SVM

In [ ]:
Sigmoid_SVM = svm.SVC(C=1.0, kernel='sigmoid')
Sigmoid_SVM.fit(Train_X,Train_Y)
# predict the labels on validation dataset
predictions_Sigmoid_SVM = Sigmoid_SVM.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_Sigmoid_SVM, Test_Y)*100)
print(classification_report(Test_Y, predictions_Sigmoid_SVM))

SVM Accuracy Score ->  92.26793467025016
              precision    recall  f1-score   support

           0       0.92      0.92      0.92      4768
           1       0.92      0.92      0.92      4906

    accuracy                           0.92      9674
   macro avg       0.92      0.92      0.92      9674
weighted avg       0.92      0.92      0.92      9674



## Save the model

In [ ]:
# save the model to disk
import pickle
with open('svm_sigmoid_model.pkl','wb') as f:
    pickle.dump(Sigmoid_SVM,f)

In [ ]:
Polynomial_SVM = svm.SVC(C=1.0, kernel='poly', degree=3)
Polynomial_SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_Polynomial_SVM = Polynomial_SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_Polynomial_SVM, Test_Y)*100)

SVM Accuracy Score ->  86.19794483504597
